In [1]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

# Função para calcular a entropia de Shannon
def shannon_entropy_scipy(seq_str):
    values, counts = np.unique(list(seq_str), return_counts=True)
    probabilities = counts / counts.sum()
    return entropy(probabilities, base=2)

# Caminho do arquivo
file_path = "3-seqbinaria_participantes.csv"
output_path = "resultado_entropia_participantes.csv"

try:
    df = pd.read_csv(file_path, sep="\t", header=None, names=['Candidato', 'Mediana', 'Sequencia'])

    results = []

    for _, row in df.iterrows():
        seq_str = row['Sequencia']
        seq_len = len(seq_str)
        seq_binaria = np.array(list(seq_str), dtype=int)
        n_1s = np.sum(seq_binaria)
        n_0s = seq_len - n_1s
        ent = shannon_entropy_scipy(seq_str)

        # Classificação conforme as três faixas fornecidas
        if ent >= 0.85:
            interpretacao = "Alta aleatoriedade"
        elif 0.3 <= ent <= 0.5:
            interpretacao = "Aleatoriedade moderada"
        else:
            interpretacao = "Baixa aleatoriedade"

        results.append({
            'candidato': row['Candidato'],
            'mediana': row['Mediana'],
            'tamanho_sequencia': seq_len,
            'n_1s': n_1s,
            'n_0s': n_0s,
            'entropia': round(ent, 4),
            'sequencia': seq_str,
            'interpretacao': interpretacao
        })

    df_results = pd.DataFrame(results)

    # Resumo
    resumo = df_results['interpretacao'].value_counts().reset_index()
    resumo.columns = ['Interpretação', 'Quantidade']
    resumo['Faixa de Entropia'] = resumo['Interpretação'].map({
        'Alta aleatoriedade': 'Entropia ≥ 0,85',
        'Aleatoriedade moderada': '0,30 ≤ Entropia ≤ 0,50',
        'Baixa aleatoriedade': 'Entropia < 0,30 ou entre 0,50 e 0,85'
    })
    resumo['Percentual (%)'] = (resumo['Quantidade'] / len(df_results) * 100).round(2)

    # Ordenação e exibição
    ordem = ['Alta aleatoriedade', 'Aleatoriedade moderada', 'Baixa aleatoriedade']
    resumo['ordem'] = resumo['Interpretação'].apply(lambda x: ordem.index(x))
    resumo = resumo.sort_values('ordem').drop(columns='ordem')
    resumo = resumo[['Interpretação', 'Faixa de Entropia', 'Quantidade', 'Percentual (%)']]

    print(resumo.to_string(index=False))

    # Salva o CSV
    df_results.to_csv(output_path, index=False)
    print()
    print()
    print(f"Arquivo '{output_path}' salvo com sucesso.")

except FileNotFoundError:
    print(f"Erro: o arquivo '{file_path}' não foi encontrado.")

except pd.errors.EmptyDataError:
    print(f"Erro: o arquivo '{file_path}' está vazio ou mal formatado.")

except Exception as e:
    print(f"Ocorreu um erro ao processar os dados: {e}")

     Interpretação Faixa de Entropia  Quantidade  Percentual (%)
Alta aleatoriedade   Entropia ≥ 0,85          55           100.0


Arquivo 'resultado_entropia_participantes.csv' salvo com sucesso.
